In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import cvxpy as cp
from math import ceil

data = pd.read_excel('data.xlsx', header=None).values.flatten()

In [2]:
requirements = data
requirements

array([ 11,   5,   4,   7,  16,   6,   5,   7,  13,   6,   5,   7,  12,
         5,   4,   6,   9,   5,   5,  11,  29,  21,  17,  20,  27,  13,
         9,  10,  16,   6,   5,   7,  11,   5,   5,   6,  12,   7,   7,
        10,  15,  10,   9,  11,  15,  10,  10,  16,  26,  21,  23,  36,
        50,  45,  45,  49,  57,  43,  40,  44,  52,  43,  42,  45,  52,
        41,  39,  41,  48,  35,  34,  35,  42,  34,  36,  43,  55,  48,
        54,  65,  80,  70,  74,  85, 101,  89,  88,  90, 100,  87,  88,
        89, 104,  89,  89,  90, 106,  96,  94,  99, 109,  99,  96, 102],
      dtype=int64)

In [3]:
total_week = 104
requirements = requirements[0:total_week]
K = 20
waste_rate = 0.1

In [4]:
working_hands = cp.Parameter(total_week)
waste_hands = cp.Parameter(total_week)
working_hands.value = 4 * requirements
waste_hands.value = np.array([
    4 * round(x * waste_rate) if x % 1 != 0.5 else ceil(4 * x * waste_rate)
    for x in requirements
])

teaching_hands = cp.Variable(total_week, integer=True)
resting_hands = cp.Variable(total_week, integer=True)
new_hands = cp.Variable((3, total_week), integer=True)

# 机械手购买的分段点
hand_price = cp.Parameter((3, 1))
hand_price.value = np.array([100, 90, 80]).reshape(3, 1)

hand_y = cp.Variable((3, total_week), boolean=True)


In [5]:
working_hands.value, waste_hands.value

(array([ 44,  20,  16,  28,  64,  24,  20,  28,  52,  24,  20,  28,  48,
         20,  16,  24,  36,  20,  20,  44, 116,  84,  68,  80, 108,  52,
         36,  40,  64,  24,  20,  28,  44,  20,  20,  24,  48,  28,  28,
         40,  60,  40,  36,  44,  60,  40,  40,  64, 104,  84,  92, 144,
        200, 180, 180, 196, 228, 172, 160, 176, 208, 172, 168, 180, 208,
        164, 156, 164, 192, 140, 136, 140, 168, 136, 144, 172, 220, 192,
        216, 260, 320, 280, 296, 340, 404, 356, 352, 360, 400, 348, 352,
        356, 416, 356, 356, 360, 424, 384, 376, 396, 436, 396, 384, 408],
       dtype=int64),
 array([ 4,  0,  0,  4,  8,  4,  0,  4,  4,  4,  0,  4,  4,  0,  0,  4,  4,
         0,  0,  4, 12,  8,  8,  8, 12,  4,  4,  4,  8,  4,  0,  4,  4,  0,
         0,  4,  4,  4,  4,  4,  8,  4,  4,  4,  8,  4,  4,  8, 12,  8,  8,
        16, 20, 16, 16, 20, 24, 16, 16, 16, 20, 16, 16, 16, 20, 16, 16, 16,
        20, 16, 12, 16, 16, 12, 16, 16, 24, 20, 20, 24, 32, 28, 28, 32, 40,
        36, 36

In [6]:
cons = [
    resting_hands[0] + teaching_hands[0] + working_hands[0] == 50,
    K * teaching_hands >= cp.sum(new_hands, axis=0),

    teaching_hands >= 0,
    resting_hands >= 0,
    new_hands >= 0,

    # 分段函数引入的约束
    # hand_y >= 0,
    # hand_y <= 1,
    20 * hand_y[1] <= new_hands[0],
    new_hands[0] <= 20 * hand_y[0],
    20 * hand_y[2] <= new_hands[1],
    new_hands[1]  <= 20 * hand_y[1],
    new_hands[2] <= 10000 * hand_y[2]
]

In [7]:
for t in range(total_week - 1):
    cons.append(resting_hands[t + 1] >= working_hands[t] - waste_hands[t]),
    cons.append(resting_hands[t + 1] + working_hands[t + 1] + teaching_hands[t + 1] == \
        resting_hands[t] + working_hands[t] + teaching_hands[t] + cp.sum(new_hands, axis=0)[t] - waste_hands[t]
        )

In [8]:
obj = cp.Minimize(
    cp.sum(cp.multiply(new_hands, hand_price))\
    + 5 * cp.sum(resting_hands)\
    + 10 * cp.sum(teaching_hands)\
    + 10 * cp.sum(new_hands)
)

In [9]:
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.CPLEX, verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 03 09:10:23 PM: Your problem has 832 variables, 216 constraints, and 211 parameters.
(CVXPY) May 03 09:10:23 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 03 09:10:23 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 03 09:10:23 PM: Compiling problem (target solver=CPLEX).
(CVXPY) May 03 09:10:23 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> CPLEX
(CVXPY) May 03 09:10:23 PM: Applying reduction CvxAttr2Constr
(CVXPY) May 03 09:10:23 PM: Applying reduction Qp2Symb

Tried aggregator 3 times.
MIP Presolve eliminated 729 rows and 107 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 72 substitutions.
Reduced MIP has 550 rows, 653 columns, and 1918 nonzeros.
Reduced MIP has 206 binaries, 447 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.47 ticks)
Probing fixed 0 vars, tightened 160 bounds.
Probing time = 0.00 sec. (0.56 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 4 coefficients.
Reduced MIP has 550 rows, 653 columns, and 1918 nonzeros.
Reduced MIP has 206 binaries, 447 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.15 ticks)
Probing fixed 0 vars, tightened 16 bounds.
Probing time = 0.00 sec. (0.56 ticks)
Clique table members: 103.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 20 threads.
Root relaxation solution time = 0.00 sec. (1.83 ticks)

        Nodes                                  

306054.99999998085

In [10]:
working_hands.value, np.array(resting_hands.value, dtype=np.int32), waste_hands.value

(array([ 44,  20,  16,  28,  64,  24,  20,  28,  52,  24,  20,  28,  48,
         20,  16,  24,  36,  20,  20,  44, 116,  84,  68,  80, 108,  52,
         36,  40,  64,  24,  20,  28,  44,  20,  20,  24,  48,  28,  28,
         40,  60,  40,  36,  44,  60,  40,  40,  64, 104,  84,  92, 144,
        200, 180, 180, 196, 228, 172, 160, 176, 208, 172, 168, 180, 208,
        164, 156, 164, 192, 140, 136, 140, 168, 136, 144, 172, 220, 192,
        216, 260, 320, 280, 296, 340, 404, 356, 352, 360, 400, 348, 352,
        356, 416, 356, 356, 360, 424, 384, 376, 396, 436, 396, 384, 408],
       dtype=int64),
 array([  5,  40,  44,  30,  24,  56,  56,  47,  24,  48,  48,  39,  24,
         48,  52,  44,  28,  40,  39,  19,  90, 111, 119,  98,  72, 116,
        128, 120,  92, 124, 124, 116,  96, 116, 116, 112,  84, 100,  96,
         80,  56,  68,  68,  55,  40,  52,  46,  36,  88,  96,  76,  84,
        184, 184, 164, 228, 176, 208, 204, 169, 179, 195, 183, 152, 164,
        188, 179, 154, 148, 1

In [11]:
np.array(hand_y.value, dtype=np.int32)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0

In [15]:
list(zip(*new_hands.value))

[(14.0, 0.0, 0.0),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0),
 (20.0, 12.0, -0.0),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0),
 (4.0, 0.0, -0.0),
 (0.0, 0.0, -0.0),
 (0.0, 0.0, -0.0),
 (0.0, 0.0, -0.0),
 (8.0, 0.0, -0.0),
 (0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (0.0, -0.0, -0.0),
 (0.0, 0.0, -0.0),
 (0.0, 0.0, -0.0),
 (0.0, 0.0, 0.0),
 (10.999999999983954, 3.1374984894781436e-11, -4.55243802927865e-11),
 (19.999999999996657, 19.999999999996657, 100.0),
 (1.2907811804891677e-11, 1.2907811804891677e-11, 1.1063838689907036e-11),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 0.0),
 (9.0, 0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, 0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (-0.0, 0.0, -0.0),
 (0.0, -0.0, -0.0),
 (0.0, -0.0, -0.0),
 (-0.0, -0.0, -0.0),
 (0.0, -0.0, -0.0),
 (0.0, -0.0, -0.0),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, -0.0),
 (0.0, 0.0, 2.6239380020669017e-13),
 (3.